# Прогнозированние. Сравниваем модели.

> 🚀 В этой практике нам понадобятся: `numpy==1.26.4, pandas==1.5.3, matplotlib==3.10.3, seaborn==0.13.2` 

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4 pandas==1.5.3 matplotlib==3.10.3 seaborn==0.13.2` 


## Содержание

* [Загрузка предсказаний](#Загрузка-предсказаний)
* [Расчёт метрик](#Расчёт-метрик)
* [Заключение](#Заключение)
* [Вопросы для закрпеления](#Вопросы-для-закрпеления)


Наконец мы добрались до главного вопроса, какая же модель лучше всех предсказывает будущее пышек. 

In [ ]:
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = [10, 7]
plt.style.use('seaborn-v0_8')

import seaborn as sns
sns.set(style="darkgrid")

import ipywidgets as widgets
import numpy as np 
import pandas as pd 

## Загрузка предсказаний

Начинаем с загрузки предсказаний от всех наших моделей.

In [ ]:
prediction_dpath = Path("forecast_predictions")

holt_winters_fpath = prediction_dpath / "holt_winters_predictions.csv"
holt_winters_pred_df = pd.read_csv(holt_winters_fpath, parse_dates=["timestamp"], index_col=0)
holt_winters_pred_df = holt_winters_pred_df.set_index("timestamp")
print(f"Holt Winters Data Shape: {holt_winters_pred_df.shape}")

linear_reg_fpath = prediction_dpath / "linear_regression_predictions.csv"
linear_reg_pred_df = pd.read_csv(linear_reg_fpath, parse_dates=["timestamp"], index_col=0).set_index("timestamp")
print(f"Linear Regression Data Shape: {linear_reg_pred_df.shape}")

lgbm_fpath = prediction_dpath / "lgbm_predictions.csv"
lgbm_pred_df = pd.read_csv(lgbm_fpath, parse_dates=["timestamp"], index_col=0).set_index("timestamp")
print(f"LGBM Data Shape: {lgbm_pred_df.shape}")

Итак предсказания у нас есть, но прежде чем считать метрики и анализировать результаты давайте снова вернёмся к вопросу пропущенных значений. 

Как вы помните, в тестовой выборке они у нас были, и мы с ними в предобработке ничего не делали. Ибо тестовая выборка всегда неприкосновенна. Однако, чтобы посчитать метрики нам придётся её немного потрогать. А именно: убрать строки, где в целевой колонке пропущено значение. Использовать такие строки для расчёта метрик бессмысленно, т.к. никто не знает, какое истинное значение там должно быть. 

In [ ]:
holt_winters_pred_df = holt_winters_pred_df.dropna(subset=["y_test"])
linear_reg_pred_df = linear_reg_pred_df.dropna(subset=["y_test"])
lgbm_pred_df = lgbm_pred_df.dropna(subset=["y_test"])

holt_winters_pred_df.shape, linear_reg_pred_df.shape, lgbm_pred_df.shape 

Сделали, теперь перейдём к коду метрик.

## Расчёт метрик

Для оценки моделей будем использовать сразу пачку метрик. Часть из них вы уже встречали, это MAE, MSE, RMSE. Тут каких-то хитростей нет - то же, что и в "сырой" регрессии. 

Однако также добавим их модицификации с весами. В этом случае, если модель предсказала меньше, чем целевое значение, будем штрафовать её сильнее. Если модель предсказала больше, то оставим значение ошибки как есть. 

На практике часто так бывает, что бизнесу выгоднее сделать больше продукции, чем если у них её не хватит. Поэтому будем исходить из таких принципов. 

Также добавим ещё пару новых метрик: 

* MAPE 
* MPE
* SMAPE

---

**MAPE** (Mean Absolute Percentage Error) - средняя **абсолютная** процентная ошибка. Метрика показывает насколько в среднем предсказания модели отличаются от реальных значений, в процентах. 

$$ MAPE = \frac{1}{n} \sum_{i=1}^{n} |\frac{y_{true}-y_{pred}}{y_{true}}| * 100\% $$

✅ Метрика интуитивно понятна 
  
✅ Безразмерна, можно сравнивать на разных данных 

⭕ Не работает, если в знаменателе `y_true = 0` 

⭕ Чрезмерно штрафует ошибки на малых значениях 

<details>
    <summary>🤓 Пример про штраф [Нажми на меня]</summary>

Например, если `𝑦_true = 1`, а предсказание 2, то 

$$ \frac{1 - 2}{1} = 100\% $$

Хотя абсолютная ошибка всего 1. Может ввести в заблуждение. 

</details>

⭕ Несимметрична 

<details>
    <summary>🤓 Пример про несимметричность [Нажми на меня]</summary>

1. `y_true = 90`, `y_pred = 100` 

$$ \frac{90 - 100}{90} = |\frac{-10}{90}| = 11.1\% $$

2. `y_true = 110`, `y_pred = 100` 

$$ \frac{110 - 100}{110} = |\frac{10}{110}| = 9.1\% $$

И там, и там абсолютное значение ошибки по модулю = 10. Но проявляется несимметричность, т.е. метрика зависит от величины истинного значения. 

Это может привести к тому, что модель может казаться хуже, чем есть, если часто ошибается на маленьких числах — ведь процент ошибки у них получается высоким. 

</details>

---

**SMAPE** (Symmetric Mean Absolute Percentage Error) - симметричная MAPE (более стабильная альтернатива). 

$$ MAPE = \frac{1}{n} \sum_{i=1}^{n} \frac{|y_{true}-y_{pred}|}{(|y_{true}| + |y_{pred}|)/2}| * 100% $$

✅ Не уходит в бесконечность при `y_true = 0` 

✅ Симметричность 

⭕ Есть проблемы с большими значениями. Если `y_true` и `y_pred` сильно отличаются друг от друга, то значение `SMAPE` может быть очень высоким. 

⭕ Не всегда легко интерпретировать. Надо быть аккуратным

---


**MPE** (Mean Percentage Error) - средняя процентная ошибка (со **знаком**). Эта метрика показывает, насколько в среднем предсказания модели отклоняются от реальных значений в процентах. 

$$ MPE = \frac{1}{n} \sum_{i=1}^{n} \frac{y_{true}-y_{pred}}{y_{true}} * 100% $$

✅ Легко интерпретировать

✅ Учитывает как положительные, так и отрицательные значения 

✅ Может быть полезна в задаче с большим разнообразием значений

⭕ Не работает, если `y_true = 0`

⭕ Смещение. Если в данных присутствуют как положительные, так и отрицательные значения, значение метрики может быть смещено в сторону одной из ошибок. Например, если большинство ошибок положительные, то метрика будет переоценена, и наоборот. Это может происходить из-за того, что ошибки в разные стороны могут компенсировать друг друга.

⭕ Чувствительность к выбросам. Большие отклонения от истинных значений будут сильно влиять на итоговое значение метрики.

---


| Метрика  | Когда использовать                                                  | Когда НЕ использовать                                  | 
|----------|---------------------------------------------------------------------|--------------------------------------------------------|
| MAPE     | - когда нужна относительная ошибка в %                              | - в данных есть нули                                   |
|          | - когда важна интерпретируемость                                    | - где важны экстремальные значения                     |
|          | - где отклонения с % - критически важны (продажи, запасы и т.д)     | - где прогнозы сильно отклоняются от истинных значений |
| MPE      | - где нужно оценить как часто модель пере- или недо- оценивает      | - в данных есть нули                                   |
|          |                                                                     | - данные сильно разнородны (шумы, выбросы)             |
| SMAPE    | - важна симметричность                                              | - где "большие" ошибки                                 |
|          | - нужна более справедливая оценка, особенно для данных близких к 0  | - где важно направление ошибок (пере-/недо-)           |    
|          | - где важен баланс ошибок, и нужно минимизировать влияние выбросов  |                                                        |

Если собрать всё воедино, то получатся следующие функции.

In [ ]:
def calculate_error(preds_df):
    # MAE, MSE, RMSE
    preds_df["error"] = preds_df["y_pred"] - preds_df["y_test"]
    preds_df["abs_error"] = np.abs(preds_df["error"])
    preds_df["sq_error"] = np.power(preds_df["error"], 2)

    preds_df["abs_error_weighted"] = np.where(preds_df["error"] < 0, 2 * np.abs(preds_df["error"]), np.abs(preds_df["error"]))
    preds_df["sq_error_weighted"] = np.where(preds_df["error"] < 0, np.power(2 * preds_df["error"], 2), np.power(preds_df["error"], 2))

    # MPE, MAPE
    preds_df["y_test_pe"] = preds_df["y_test"]
    preds_df.loc[preds_df["y_test_pe"] == 0, "y_test_pe"] = np.nan
    preds_df["perc_error"] = (preds_df["y_test_pe"] - preds_df["y_pred"]) / preds_df["y_test_pe"] * 100
    preds_df["abs_perc_error"] = preds_df["perc_error"].abs()


    preds_df["sym_abs_perc_error"] = (
        (preds_df["y_test_pe"] - preds_df["y_pred"]).abs() 
        / ((preds_df["y_test_pe"].abs() + preds_df["y_pred"].abs()) / 2)
    ) * 100

    preds_df = preds_df.drop(columns=["y_test_pe"])

    return preds_df


def calculate_metrics(preds_df):
    metrics_df = (
        preds_df
        .agg(
            {
                "abs_error": "mean", 
                "sq_error": "mean",
                "abs_error_weighted": "mean", 
                "sq_error_weighted": "mean",
                "perc_error": "mean", 
                "abs_perc_error": "mean",
                "sym_abs_perc_error": "mean",
            }
        )
        .reset_index()
    )
    metrics_df = pd.pivot_table(metrics_df, columns="index")
    metrics_df = metrics_df.rename(
        columns={
            "abs_error": "MAE",
            "sq_error": "MSE",
            "abs_error_weighted": "wMAE", 
            "sq_error_weighted": "wMSE",
            "perc_error": "MPE",
            "abs_perc_error": "MAPE",
            "sym_abs_perc_error": "SMAPE",
        }
    )
    metrics_df.columns.name = ""

    metrics_df["RMSE"] = np.sqrt(metrics_df["MSE"])
    metrics_df["wRMSE"] = np.sqrt(metrics_df["wMSE"])

    return metrics_df

А теперь посчитаем метрики для каждой из моделей и соберём всё в единую табличку.

In [ ]:
holt_winters_errors = calculate_error(holt_winters_pred_df)
holt_winter_metrics = calculate_metrics(holt_winters_errors)
holt_winter_metrics["model"] = "holt_winters"

linear_reg_errors = calculate_error(linear_reg_pred_df)
linear_reg_metrics = calculate_metrics(linear_reg_errors)
linear_reg_metrics["model"] = "linear_reg"

lgbm_errors = calculate_error(lgbm_pred_df)
lgbm_metrics = calculate_metrics(lgbm_errors)
lgbm_metrics["model"] = "lgbm"

metrics_df = pd.concat((holt_winter_metrics, linear_reg_metrics, lgbm_metrics))

metrics_df.sort_values("MAE")

Если смотреть на значения MAE, то лучшие результаты показывает модель LGBM, а худшие - модель Хольта-Винтерса. 

Да и по остальным метрикам можно видеть похожую картину.

Однако важно помимо метрик ещё провести и визуальный анализ. Давайте сделаем это. 

Для полноты картины ещё загрузим обучающие данные, чтобы смотреть немного больше в прошлое.

In [ ]:
dataset_dpath = Path("ts_datasets")
train_fpath = dataset_dpath / "train.csv"

train_df = pd.read_csv(train_fpath, index_col=0, parse_dates=["timestamp"]).set_index("timestamp")
train_df.shape

In [ ]:
@widgets.interact(
    country=widgets.Dropdown(options=train_df["country"].unique()),
    store=widgets.Dropdown(options=train_df["store"].unique()),
    products=widgets.Dropdown(options=train_df["product"].unique()),
)
def show_lgbm_predictions(country: str, store: str, products: str):
    train_plot_data = train_df[
        (train_df["country"] == country)
        & (train_df["store"] == store)
        & (train_df["product"] == products)
    ]
    # оставим только 1 месяц из обучающих данных, иначе на графике плохо видно (=
    train_plot_data = train_plot_data.loc["2016-11-01":]

    holt_winters_plot_df = holt_winters_pred_df[
        (holt_winters_pred_df["country"] == country)
        & (holt_winters_pred_df["store"] == store)
        & (holt_winters_pred_df["product"] == products)
    ]

    linreg_plot_df = linear_reg_pred_df[
        (linear_reg_pred_df["country"] == country)
        & (linear_reg_pred_df["store"] == store)
        & (linear_reg_pred_df["product"] == products)
    ]

    lgbm_plot_df = lgbm_pred_df[
        (lgbm_pred_df["country"] == country)
        & (lgbm_pred_df["store"] == store)
        & (lgbm_pred_df["product"] == products)
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(train_plot_data.index, train_plot_data["target"], marker="o", label="train")

    axs.plot(holt_winters_plot_df.index, holt_winters_plot_df["y_test"], marker="o", label="true")
    axs.plot(holt_winters_plot_df.index, holt_winters_plot_df["y_pred"], marker="o", label="holt_winter_pred")
    axs.plot(linreg_plot_df.index, linreg_plot_df["y_pred"], marker="o", label="linreg_pred")
    axs.plot(lgbm_plot_df.index, lgbm_plot_df["y_pred"], marker="o", label="lgbm_pred")
    
    axs.set_xlabel("Дата")
    axs.set_ylabel("Количество пышек")
    axs.legend()

    plt.xticks(rotation=90)
    plt.show()

Итак, что мы видим: 

* LGBM тащит и корректно понимает зависимости в данных;
* Линейная регрессия в целом тоже неплохо справляется, но в конце тестового ряда её ведёт немного не туда;
* Статистические модели выступают скорее как апроксимация (усреднение). Но в целом - это их основа, так что ожидаемо. 

## Заключение

Это был последний ноутбук по теме прогнозирования. Мы с вами прошли полный пайплайн от анализа сырых данных, до оценки обученных моделей. 

Можно сказать, что мы только что решили задачу прогнозирования для небольшого стартапа с пышками. 

Хочется ещё раз подчеркнуть, что в машинном обучении, а особенно в прогнозировании, не существует какого-то абсолютно верного решения. Всё может измениться, если данные будут содержать в себе какие-то приколы. Обращайте на это внимание, и пусть в вашем доме всегда будет вдоволь пышек! 

## Вопросы для закрпеления

1. Зачем нужны взвешенные метрики MAE, MSE, RMSE? Что они такого особенного показывают? 
2. Что покажет метрика MPE в отличие от MAPE и SMAPE? 
3. Почему модель Холта-Винтерса дала худшие результаты? 
4. Можно ли посмотреть только на метрики, не проводя визуальный анализ? 